In [ ]:
from fyers_api import accessToken
from fyers_api.Websocket import ws
import pandas as pd




In [ ]:
client_id='TAZ287TZUA-100'
secret_id='PPZ5ZG1YAR'
url='https://www.google.com/'
response_type = "code"
grant_type = "authorization_code"

In [ ]:


session=accessToken.SessionModel(
    client_id=client_id,
    secret_key=secret_id,
    redirect_uri=url, 
    response_type=response_type,
    grant_type=grant_type
)

response = session.generate_authcode()
print(response)



In [ ]:
auth_code='eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkubG9naW4uZnllcnMuaW4iLCJpYXQiOjE2OTU4OTQ2MDYsImV4cCI6MTY5NTkyNDYwNiwibmJmIjoxNjk1ODk0MDA2LCJhdWQiOiJbXCJ4OjBcIiwgXCJ4OjFcIiwgXCJ4OjJcIiwgXCJkOjFcIiwgXCJkOjJcIiwgXCJ4OjFcIiwgXCJ4OjBcIl0iLCJzdWIiOiJhdXRoX2NvZGUiLCJkaXNwbGF5X25hbWUiOiJYTTA0MzQyIiwib21zIjoiSzEiLCJoc21fa2V5IjoiNWI3Mzg3ZTYxOGQzODllNGVjOGVmZjU1ODM0ZTBmMTA1NjdjYWU1ODk5YWQ5NWY3MzgxMTM3YTMiLCJub25jZSI6IiIsImFwcF9pZCI6IlRBWjI4N1RaVUEiLCJ1dWlkIjoiYjgzOWI4ODhkNTIzNDFmOWFiNjQ0OWM2MjlhMTMwMDQiLCJpcEFkZHIiOiIwLjAuMC4wIiwic2NvcGUiOiIifQ.6c7WCkY7zXuVvPsoBsbigmm5bn1Xs8-nj-4sh9GUqf4'

In [ ]:
from fyers_api import accessToken


session.set_token(auth_code)
response = session.generate_token()
print(response)



In [ ]:
access_token=response['access_token']

In [ ]:
access_token

In [ ]:
from fyers_api import fyersModel


In [ ]:
import os
fyers=fyersModel.FyersModel(client_id=client_id,token=access_token,log_path=os.getcwd())

In [ ]:
fyers

In [ ]:
data = {
    "symbol":"NSE:INFY-EQ",
    "resolution":"5",
    "date_format":"1",
    "range_from":"2023-08-30",
    "range_to":"2023-08-30",
    "cont_flag":"1"
}

response = fyers.history(data=data)

In [ ]:
response['candles']


In [ ]:
candleSticks=response['candles']

In [ ]:
closingValues=[]

for item in candleSticks:
    if(item[2]>item[3]):
        closingValues.append(item[2])
    else:
        closingValues.append(item[3])

In [ ]:
valuesToPlot={'closingValues':closingValues}
df = pd.DataFrame(valuesToPlot)

In [ ]:
# calculating RSI (gives the same values as TradingView)
# https://stackoverflow.com/questions/20526414/relative-strength-index-in-python-pandas
def RSI(series, period=14):
    delta = series.diff().dropna()
    ups = delta * 0
    downs = ups.copy()
    ups[delta > 0] = delta[delta > 0]
    downs[delta < 0] = -delta[delta < 0]
    ups[ups.index[period-1]] = np.mean( ups[:period] ) #first value is sum of avg gains
    ups = ups.drop(ups.index[:(period-1)])
    downs[downs.index[period-1]] = np.mean( downs[:period] ) #first value is sum of avg losses
    downs = downs.drop(downs.index[:(period-1)])
    rs = ups.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() / \
         downs.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() 
    return 100 - 100 / (1 + rs)


# calculating Stoch RSI (gives the same values as TradingView)
# https://www.tradingview.com/wiki/Stochastic_RSI_(STOCH_RSI) 
def StochRSI(series, period=14, smoothK=3, smoothD=3):
    # Calculate RSI 
    delta = series.diff().dropna()
    ups = delta * 0
    downs = ups.copy()
    ups[delta > 0] = delta[delta > 0]
    downs[delta < 0] = -delta[delta < 0]
    ups[ups.index[period-1]] = np.mean( ups[:period] ) #first value is sum of avg gains
    ups = ups.drop(ups.index[:(period-1)])
    downs[downs.index[period-1]] = np.mean( downs[:period] ) #first value is sum of avg losses
    downs = downs.drop(downs.index[:(period-1)])
    rs = ups.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() / \
         downs.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() 
    rsi = 100 - 100 / (1 + rs)

    # Calculate StochRSI 
    stochrsi  = (rsi - rsi.rolling(period).min()) / (rsi.rolling(period).max() - rsi.rolling(period).min())
    stochrsi_K = stochrsi.rolling(smoothK).mean()
    stochrsi_D = stochrsi_K.rolling(smoothD).mean()

    return stochrsi, stochrsi_K, stochrsi_D


# calculating Stoch RSI 
#  -- Same as the above function but uses EMA, not SMA
def StochRSI_EMA(series, period=14, smoothK=3, smoothD=3):
    # Calculate RSI 
    delta = series.diff().dropna()
    ups = delta * 0
    downs = ups.copy()
    ups[delta > 0] = delta[delta > 0]
    downs[delta < 0] = -delta[delta < 0]
    ups[ups.index[period-1]] = np.mean( ups[:period] ) #first value is sum of avg gains
    ups = ups.drop(ups.index[:(period-1)])
    downs[downs.index[period-1]] = np.mean( downs[:period] ) #first value is sum of avg losses
    downs = downs.drop(downs.index[:(period-1)])
    rs = ups.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() / \
         downs.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() 
    rsi = 100 - 100 / (1 + rs)

    # Calculate StochRSI 
    stochrsi  = (rsi - rsi.rolling(period).min()) / (rsi.rolling(period).max() - rsi.rolling(period).min())
    stochrsi_K = stochrsi.ewm(span=smoothK).mean()
    stochrsi_D = stochrsi_K.ewm(span=smoothD).mean()

    return stochrsi, stochrsi_K, stochrsi_D

In [ ]:
StochRSI(df['closingValues'])

In [ ]:
import pandas as pd
import numpy as np


def rsi_tradingview(ohlc: pd.DataFrame, period: int = 14, round_rsi: bool = True):
    """ Implements the RSI indicator as defined by TradingView on March 15, 2021.
        The TradingView code is as follows:
        //@version=4
        study(title="Relative Strength Index", shorttitle="RSI", format=format.price, precision=2, resolution="")
        len = input(14, minval=1, title="Length")
        src = input(close, "Source", type = input.source)
        up = rma(max(change(src), 0), len)
        down = rma(-min(change(src), 0), len)
        rsi = down == 0 ? 100 : up == 0 ? 0 : 100 - (100 / (1 + up / down))
        plot(rsi, "RSI", color=#8E1599)
        band1 = hline(70, "Upper Band", color=#C0C0C0)
        band0 = hline(30, "Lower Band", color=#C0C0C0)
        fill(band1, band0, color=#9915FF, transp=90, title="Background")

    :param ohlc:
    :param period:
    :param round_rsi:
    :return: an array with the RSI indicator values
    """

    delta = ohlc["closingValues"].diff()

    up = delta.copy()
    up[up < 0] = 0
    up = pd.Series.ewm(up, alpha=1/period).mean()

    down = delta.copy()
    down[down > 0] = 0
    down *= -1
    down = pd.Series.ewm(down, alpha=1/period).mean()

    rsi = np.where(up == 0, 0, np.where(down == 0, 100, 100 - (100 / (1 + up / down))))

    return np.round(rsi, 2) if round_rsi else rsi

In [ ]:
rsi_tradingview(df)

In [ ]:
import pandas as pd




def stoch_rsi_tradingview(ohlc: pd.DataFrame, period=14, smoothK=3, smoothD=3):
    """ Calculating Stochastic RSI (gives the same values as TradingView as of March 20, 2021.
        smoothK = input(3, "K", minval=1)
        smoothD = input(3, "D", minval=1)
        lengthRSI = input(14, "RSI Length", minval=1)
        lengthStoch = input(14, "Stochastic Length", minval=1)
        src = input(close, title="RSI Source")
        rsi1 = rsi(src, lengthRSI)
        k = sma(stoch(rsi1, rsi1, rsi1, lengthStoch), smoothK)
        d = sma(k, smoothD)

    :param ohlc:
    :param period:
    :param smoothK:
    :param smoothD:
    :return:
    """
    # Calculate RSI
    rsi = rsi_tradingview(ohlc, period=period, round_rsi=False)

    # Calculate StochRSI
    rsi = pd.Series(rsi)
    stochrsi  = (rsi - rsi.rolling(period).min()) / (rsi.rolling(period).max() - rsi.rolling(period).min())
    stochrsi_K = stochrsi.rolling(smoothK).mean()
    stochrsi_D = stochrsi_K.rolling(smoothD).mean()

    return round(rsi, 2), round(stochrsi_K * 100, 2), round(stochrsi_D * 100, 2)

In [ ]:
stoch_rsi_tradingview(df)

In [ ]:
df['closingValues']
small_lookback = 5 
long_lookback = 20

df['mean'] = df['closingValues'].rolling(window = small_lookback ,min_periods = small_lookback).mean()
df['mean_Long'] = df['closingValues'].rolling(window = long_lookback ,min_periods = long_lookback).mean()

In [ ]:
df

In [ ]:
df['SMA_diff'] = df['mean'] - df['mean_Long']
df['zero_line'] = 0

df['Signal'] = 0
dfForBuyingAndClosing={}
buyingPrice=[]
sellingPrice=[]

In [ ]:
for index, row in df.iterrows(): 
    if index != 0 :
        if df['SMA_diff'][index-1]<0 and df['SMA_diff'][index] >=0: 
            df['Signal'][index] =  "SELL"
            sellingPrice.append(df['closingValues'][index])    
            print("Selling at "+str(df['closingValues'][index]))
        elif df['SMA_diff'][index-1]>0 and df['SMA_diff'][index] <=0:
            print("Buying at "+str(df['closingValues'][index]))
            buyingPrice.append(df['closingValues'][index])
            df['Signal'][index] = "BUY"

In [ ]:
df

In [ ]:
dfForBuyingAndClosing['buying']=buyingPrice
dfForBuyingAndClosing['selling']=sellingPrice

In [ ]:
dfForBuyingAndClosing


In [ ]:
dfGraph=pd.DataFrame(dfForBuyingAndClosing)

In [ ]:
dfGraph

In [ ]:
dfGraph.plot()

In [ ]:
dfForBuyingAndClosing

In [ ]:
totalBuying=sum(dfForBuyingAndClosing['buying'])
totalSelling=sum(dfForBuyingAndClosing['selling'])

In [ ]:
profitPercentage=(totalSelling-totalBuying)*100

In [ ]:
profitPercentage

In [ ]:
totalSelling-totalBuying

In [ ]:
array_for_database=[]
fyers.get_profile()


In [ ]:
df

In [ ]:
plottingValues={'closingValues':df['closingValues'].tolist(),'mean':df['mean'].tolist(),'meanLong':df['mean_Long'].tolist()}
dfToShow=pd.DataFrame(plottingValues)
dfToShow.plot()

In [ ]:
plottingValues

In [ ]:
symbols=["NSE:INFY-EQ","NSE:LT-EQ"]
presentCandleStick={}
candleSticks={}
timeStamp=-420
open=-420
high=-420
low=-420
timeStampClose=-420
for symbol in symbols:
    presentCandleStick[symbol]=[]
    candleSticks[symbol]=[]


In [ ]:
import csv

In [ ]:
with open('lastTradedPrices.csv', 'w', newline='') as file:
    writer = csv.writer(file)
     
    writer.writerow(['EPOCH','Stock Name','ltp'])

In [12]:


import csv
from fyers_api.Websocket import ws
symbols=["NSE:INFY-EQ","NSE:LT-EQ"]
presentCandleStick={}
candleSticks={}
candleStickLength=1

for symbol in symbols:
    presentCandleStick[symbol]=[-420]
    candleSticks[symbol]=[]
print(presentCandleStick)
print(candleSticks)
def run_process_order_update(access_token):
    data_type = "symbolData"
    print(access_token)
    fs = ws.FyersSocket(access_token=access_token,log_path=os.getcwd())
    fs.websocket_data = custom_message
    fs.subscribe(symbol=symbols,data_type=data_type)
    fs.keep_running()


def custom_message(msg):
    
    global candleSticks
    global presentCandleStick
    global candleStickLength
    #print(msg[0])
    try:
            if(presentCandleStick[msg[0]['symbol']][0]==-420):
                presentCandleStick[msg[0]['symbol']].append(msg[0]['ltp'])
                presentCandleStick[msg[0]['symbol']].append(msg[0]['ltp'])
                presentCandleStick[msg[0]['symbol']].append(msg[0]['ltp'])
                presentCandleStick[msg[0]['symbol']][0]=msg[0]['timestamp']+(candleStickLength*60)
            else:
                if(msg[0]['timestamp']>=presentCandleStick[msg[0]['symbol']][0]):
                    presentCandleStick[msg[0]['symbol']].append(msg[0]['ltp'])
                    if(msg[0]['ltp']>presentCandleStick[msg[0]['symbol']][2]):
                        presentCandleStick[msg[0]['symbol']][2]=msg[0]['ltp']
                    if(msg[0]['ltp']<presentCandleStick[msg[0]['symbol']][3]):
                        presentCandleStick[msg[0]['symbol']][3]=msg[0]['ltp']
                    candleSticks[msg[0]['symbol']].append(presentCandleStick[msg[0]['symbol']])
                    presentCandleStick[msg[0]['symbol']]=[-420]
                    print(candleSticks)
                else:
                    if(msg[0]['ltp']>presentCandleStick[msg[0]['symbol']][2]):
                        presentCandleStick[msg[0]['symbol']][2]=msg[0]['ltp']
                    if(msg[0]['ltp']<presentCandleStick[msg[0]['symbol']][3]):
                        presentCandleStick[msg[0]['symbol']][3]=msg[0]['ltp']
    except Exception as e: print(e)

    

    


def main():
    token_to_use = f"{client_id}:{access_token}"
    run_process_order_update(token_to_use)


if __name__ == '__main__':
  main()






{'NSE:INFY-EQ': [-420], 'NSE:LT-EQ': [-420]}
{'NSE:INFY-EQ': [], 'NSE:LT-EQ': []}
TAZ287TZUA-100:eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkuZnllcnMuaW4iLCJpYXQiOjE2OTU4OTQ2MjMsImV4cCI6MTY5NTk0NzQyMywibmJmIjoxNjk1ODk0NjIzLCJhdWQiOlsieDowIiwieDoxIiwieDoyIiwiZDoxIiwiZDoyIiwieDoxIiwieDowIl0sInN1YiI6ImFjY2Vzc190b2tlbiIsImF0X2hhc2giOiJnQUFBQUFCbEZVeGZXd01XdVpCN0NLMVltLU9mSlMtWFNJelZLTnB3YlBWa0tzOXpuUllDMGhKY3hVNVgxNnZMYUx2MWxIMXdtZVFsVVN3YWxyNmpxdFJCV2R4NVZiekUtLWNuanpTb2NkUnNxTkltUVI5aGl0az0iLCJkaXNwbGF5X25hbWUiOiJNT0hBTU1FRCBSQVlZQU4gU0FJVCIsIm9tcyI6IksxIiwiaHNtX2tleSI6IjViNzM4N2U2MThkMzg5ZTRlYzhlZmY1NTgzNGUwZjEwNTY3Y2FlNTg5OWFkOTVmNzM4MTEzN2EzIiwiZnlfaWQiOiJYTTA0MzQyIiwiYXBwVHlwZSI6MTAwLCJwb2FfZmxhZyI6Ik4ifQ.Hu7trKcN5Lu3ORbKvjPd6QDgXHCHSj4zyiPWPoZjjDE
{'NSE:INFY-EQ': [[1695894779, 1439.65, 1440.55, 1439.6, 1439.9]], 'NSE:LT-EQ': []}
{'NSE:INFY-EQ': [[1695894779, 1439.65, 1440.55, 1439.6, 1439.9]], 'NSE:LT-EQ': [[1695894779, 3007.95, 3013.2, 3007.95, 3012.45]]}
{'NSE:INFY-EQ': [[

In [ ]:
import pandas as pd
import numpy as np

# Define the candlestick data
candlestick_data = [
    [1691034300, 1350.0, 1356.4, 1349.0, 1356.05, 358630],
    # Add more candlestick data if needed
]

# Extract the closing prices from the candlestick data
closing_prices = [candle[4] for candle in candlestick_data]

def stochastic_rsi(data, length_rsi, length_stoch, smooth_k, smooth_d):
    rsi = pd.Series(data).rsi(length=length_rsi)
    rsi_k = pd.Series(rsi).rolling(window=length_stoch).mean()
    rsi_d = pd.Series(rsi_k).rolling(window=smooth_k).mean()
    return rsi_k, rsi_d

# Define your input parameters
lengthRSI = 14
lengthStoch = 14
smoothK = 3
smoothD = 3

# Calculate Stochastic RSI values
k, d = stochastic_rsi(closing_prices, lengthRSI, lengthStoch, smoothK, smoothD)

# Print the calculated Stochastic RSI values
print("Stochastic RSI K values:")
print(k)
print("\nStochastic RSI D values:")
print(d)


In [ ]:


import csv
from fyers_api.Websocket import ws
symbols=["NSE:INFY-EQ","NSE:LT-EQ"]
presentCandleStick={}
candleSticks={}
timeStamp=-420
open=-420
high=-420
low=-420
timeStampClose=-420
for symbol in symbols:
    presentCandleStick[symbol]=[]
    candleSticks[symbol]=[]
print(presentCandleStick)
print(candleSticks)
def run_process_order_update(access_token):
    data_type = "symbolData"
    print(access_token)
    fs = ws.FyersSocket(access_token=access_token,log_path=os.getcwd())
    fs.websocket_data = custom_message
    fs.subscribe(symbol=symbols,data_type=data_type)
    fs.keep_running()


def custom_message(msg):
    global timeStamp
    global open
    global high
    global low
    global close
    global timeStampClose
    print(msg[0])
    try:
            if(timeStamp==-420):
                print('First Candlestick:'+str(msg[0]['timestamp']))
                timeStamp=msg[0]['timestamp']
                open=msg[0]['ltp']
                high=msg[0]['ltp']
                low=msg[0]['ltp']
                timeStampClose=timeStamp+(5*60)
            else:
                if(msg[0]['timestamp']>=timeStampClose):
                    close=msg[0]['ltp']
                    if(msg[0]['ltp']>high):
                        high=msg[0]['ltp']
                    if(msg[0]['ltp']<low):
                        low=msg[0]['ltp']
                    print('CandleStick at:'+str(msg[0]['timestamp'])+' '+str(open)+' '+str(high)+' '+str(low)+' '+str(close))
                    
                    timeStamp=-420
                    timeStampClose=-420
                    open=-420
                    high=-420
                    low=-420
                    close=-420
                    

                else:
                    if(msg[0]['ltp']>high):
                        high=msg[0]['ltp']
                    if(msg[0]['ltp']<low):
                        low=msg[0]['ltp']
    except Exception as e: print(e)

    

    


def main():
    token_to_use = f"{client_id}:{access_token}"
    run_process_order_update(token_to_use)


if __name__ == '__main__':
  main()




